In [26]:
import re
import json
from jsonpath_ng import jsonpath, parse
import pandas as pd

In [29]:
data_dir = './data/uplink_long/'
nruejsonfile = data_dir+'nruejourneys.json'
gnbjsonfile = data_dir+'gnbjourneys.json'
irtt_server_jsonfile = data_dir+'se_12-1-1-2_59340_20231213_011637.json'
irtt_client_jsonfile = data_dir+'cl_12-1-1-2_59340_20231213_011637.json'

In [ ]:
python tools/rdtsctots.py 2_results/latseq.30102023_152306.lseq > 2_results/gnb.lseq
python ul_postprocess_gnb.py 2_results/gnb.lseq > 2_results/gnbjourneys.json

In [ ]:
python tools/rdtsctots.py 2_results/latseq.30102023_152306.lseq > 2_results/nrue_tmp.lseq
tac 2_results/nrue_tmp.lseq > 2_results/nrue.lseq
python ul_postprocess_nrue.py 2_results/nrue.lseq > 2_results/nruejourneys.json

In [27]:
import json
from jsonpath_ng import jsonpath, parse
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.tools as tls  # Import plotly.tools
from plotly.subplots import make_subplots

IRTT_TIME_MARGIN = 0.0010 # 500us

def flatten_dict(d, parent_key='', sep='.'):
    items = {}
    for key, value in d.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.update(flatten_dict(value, new_key, sep=sep))
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    items.update(flatten_dict(item, f"{new_key}{sep}{i}", sep=sep))
                else:
                    items[f"{new_key}{sep}{i}"] = item
        else:
            items[new_key] = value
    return items

def closest_irtt_entry(ue_timestamp, gnb_timestamp, irtt_dict):
    for seqno in irtt_dict:
        entry = irtt_dict[seqno]
        send_timestamp, receive_timestamp = entry['send.timestamp'], entry['receive.timestamp']
        if send_timestamp <  ue_timestamp and send_timestamp + IRTT_TIME_MARGIN >=  ue_timestamp:
            import pdb; pdb.set_trace()
            if gnb_timestamp <  receive_timestamp and gnb_timestamp + IRTT_TIME_MARGIN >=  receive_timestamp:
                return entry
        #elif send_timestamp > ue_timestamp + IRTT_TIME_MARGIN:
        #    print("Not found corresponding irtt entry")
        #    return None
    return None


In [28]:
# Load the first JSON file (nruejourneys.json)
with open(nruejsonfile, 'r') as nruejourneys_file:
    uejourneys_data = json.load(nruejourneys_file)

# Load the second JSON file (gnbjourneys.json)
with open(gnbjsonfile, 'r') as gnbjourneys_file:
    gnbjourneys_data = json.load(gnbjourneys_file)
    
# Load the second JSON file
with open(irtt_server_jsonfile, 'r') as irtt_server_file:
    irtt_server_data = json.load(irtt_server_file)['oneway_trips']


# Create dicts
gnbjourneys_dict = {entry['gtp.out']['sn']: entry for entry in gnbjourneys_data}

uejourneys_dict = {entry['rlc.queue']['segments'][0]['rlc.txpdu']['sn']: entry for entry in uejourneys_data}

irtt_dict = {entry['seqno']: {'send.timestamp': entry['timestamps']['client']['send']['wall']/1.0e9, 'receive.timestamp': entry['timestamps']['server']['receive']['wall']/1.0e9} for entry in irtt_server_data if entry['timestamps']['client']['send'].get('wall')!=None}

In [30]:
ADD_IRTT_TIMESTAMPS = False
journey_parquet_file = data_dir + 'journeys.parquet'
# combine ue and gnb
combined_dict = {}
processed_entries = 0
for uekey in uejourneys_dict:
    ue_entry = uejourneys_dict[uekey]
    if uekey in gnbjourneys_dict:
        gnb_entry = gnbjourneys_dict[uekey]
        # find the closest irtt send and receive timestamps
        ue_timestamp, gnb_timestamp = ue_entry['ip.in']['timestamp'], gnb_entry['gtp.out']['timestamp']
        if ADD_IRTT_TIMESTAMPS==False:
            combined_dict[uekey] = flatten_dict(ue_entry, parent_key='', sep='.') | flatten_dict(gnb_entry, parent_key='', sep='.')
        else:
            irtt_entry = closest_irtt_entry(ue_entry['ip.in']['timestamp'], gnb_entry['gtp.out']['timestamp'], irtt_dict)
            if irtt_entry!=None:
                combined_dict[uekey] = flatten_dict(irtt_entry, parent_key='', sep='.') | flatten_dict(ue_entry, parent_key='', sep='.') | flatten_dict(gnb_entry, parent_key='', sep='.')
                processed_entries += 1
                print("processed_entries: {processed_entries}/{len(irtt_dict)}")
df = pd.DataFrame(combined_dict).T  # Transpose to have keys as columns
df.to_parquet(journey_parquet_file, engine='pyarrow')

In [31]:
! python ul_plot.py data/uplink_long/journeys.parquet data/uplink_long/res.png

2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:13 - Column Names:
2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:15 - ip.in.timestamp
2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:15 - ip.in.length
2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:15 - ip.in.PBuf
2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:15 - pdcp.cipher.timestamp
2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:15 - pdcp.cipher.length
2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:15 - pdcp.cipher.PCBuf
2023-12-13 12:21:49.176 | INFO     | __main__:read_parquet_file:15 - pdcp.pdu.timestamp
2023-12-13 12:21:49.177 | INFO     | __main__:read_parquet_file:15 - pdcp.pdu.length
2023-12-13 12:21:49.177 | INFO     | __main__:read_parquet_file:15 - pdcp.pdu.R1buf
2023-12-13 12:21:49.177 | INFO     | __main__:read_parquet_file:15 - rlc.queue.timestamp
2023-12-13 12:21:49.177 | INFO     | __main__:read_parquet

In [33]:
print(df.shape)

(179748, 100)
